In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [5]:
from google.colab import files
uploaded = files.upload()

Saving Training.zip to Training.zip


In [7]:
!ls

1JYWEFYFJCSRsVPmBfkauPQ  sample_data  Training.zip


In [10]:
!unzip Training.zip

Archive:  Training.zip
   creating: Training/00000/
  inflating: Training/00000/01153_00000.ppm  
  inflating: Training/00000/01153_00001.ppm  
  inflating: Training/00000/01153_00002.ppm  
  inflating: Training/00000/01160_00000.ppm  
  inflating: Training/00000/01160_00001.ppm  
  inflating: Training/00000/01160_00002.ppm  
  inflating: Training/00000/01797_00000.ppm  
  inflating: Training/00000/01797_00001.ppm  
  inflating: Training/00000/01797_00002.ppm  
  inflating: Training/00000/01798_00000.ppm  
  inflating: Training/00000/01798_00001.ppm  
  inflating: Training/00000/01798_00002.ppm  
  inflating: Training/00000/01799_00000.ppm  
  inflating: Training/00000/01799_00001.ppm  
  inflating: Training/00000/01799_00002.ppm  
  inflating: Training/00000/GT-00000.csv  
   creating: Training/00001/
  inflating: Training/00001/00025_00000.ppm  
  inflating: Training/00001/00025_00001.ppm  
  inflating: Training/00001/00025_00002.ppm  
  inflating: Training/00001/00029_00000.ppm  
  

In [43]:
import glob
from skimage.transform import resize
import numpy as np 
import matplotlib
import matplotlib.pyplot as plt
import skimage.transform
import tensorflow as tf
import os
import cv2

In [44]:
def read_dataset(Directory):
  labels = []
  images = []
 
  directories = [direc for direc in os.listdir(Directory) 
                   if os.path.isdir(os.path.join(Directory, direc))]

  for direc in directories:
        label = os.path.join(Directory, direc)
        img_names = [os.path.join(label, img) 
                      for img in os.listdir(label) if img.endswith(".ppm")]
       
        for img in img_names:
          imag = cv2.imread(img)
          imag = skimage.transform.resize(imag, (75,75), mode='constant')
          images.append(imag)
          labels.append(int(direc))

  return images, labels

In [45]:
images, labels = read_dataset('Training/')
images_test, labels_test = read_dataset('Training/')

In [46]:
np_labels = np.array(labels).astype(np.float32)
np_images = np.array(images).astype(np.float32)
print("labels: ", np_labels.shape, "\nimages: ", np_images.shape)

labels:  (587,) 
images:  (587, 75, 75, 3)


In [47]:
np_test_labels = np.array(labels_test).astype(np.float32)
np_test_images = np.array(images_test).astype(np.float32)
print("images: ", np_test_images.shape, "\nlabels: ", np_test_labels.shape)

images:  (587, 75, 75, 3) 
labels:  (587,)


In [50]:
IMG_SHAPE = (75, 75, 3)
base_model = tf.keras.applications.xception.Xception(input_shape=IMG_SHAPE,include_top=False,weights='imagenet')
#base_model = tf.keras.applications.inception_resnet_v2.InceptionResNetV2(input_shape=IMG_SHAPE,include_top=False,weights='imagenet')

83689472/83683744 [==============================] - 0s 0us/step


In [51]:
feature_batch= base_model(np_images)

In [52]:
base_model.trainable = False

In [53]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

(587, 2048)


In [54]:
prediction_layer = tf.keras.layers.Dense(62,activation='linear')
dropp =tf.keras.layers.Dropout(0.4)
dropp2 =tf.keras.layers.Dropout(0.5)
prediction_layer2 = tf.keras.layers.Dense(1536,activation='relu')
#prediction_batch = prediction_layer(feature_batch_average)
#print(prediction_batch.shape)

In [55]:
model = tf.keras.Sequential([
  base_model,
  dropp,
  global_average_layer,
  dropp2,
  prediction_layer2,
  prediction_layer
])

In [56]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, 3, 3, 2048)        20861480  
_________________________________________________________________
dropout_2 (Dropout)          (None, 3, 3, 2048)        0         
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 1536)              3147264   
_________________________________________________________________
dense_2 (Dense)              (None, 62)                95294     
Total params: 24,104,038
Trainable params: 3,242,558
Non-trainable params: 20,861,480
__________________________________

In [57]:
from sklearn.model_selection import train_test_split

In [58]:
X_train, X_valid, y_train, y_valid = train_test_split(np_images, np_labels, test_size=0.05, random_state=42)

In [59]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [60]:
history = model.fit(X_train,y_train,epochs=10,validation_data=(X_valid,y_valid))

Epoch 1/10
18/18 [==============================] - 4s 83ms/step - loss: 3.1181 - accuracy: 0.3224 - val_loss: 1.4373 - val_accuracy: 0.6333
Epoch 2/10
18/18 [==============================] - 0s 23ms/step - loss: 1.2336 - accuracy: 0.6409 - val_loss: 1.0703 - val_accuracy: 0.7333
Epoch 3/10
18/18 [==============================] - 0s 22ms/step - loss: 0.7000 - accuracy: 0.7910 - val_loss: 0.8394 - val_accuracy: 0.7333
Epoch 4/10
18/18 [==============================] - 0s 23ms/step - loss: 0.4706 - accuracy: 0.8482 - val_loss: 0.9238 - val_accuracy: 0.6667
Epoch 5/10
18/18 [==============================] - 0s 22ms/step - loss: 0.3708 - accuracy: 0.8945 - val_loss: 0.9023 - val_accuracy: 0.7333
Epoch 6/10
18/18 [==============================] - 0s 23ms/step - loss: 0.2958 - accuracy: 0.8966 - val_loss: 0.6782 - val_accuracy: 0.8000
Epoch 7/10
18/18 [==============================] - 0s 22ms/step - loss: 0.2428 - accuracy: 0.9351 - val_loss: 0.7396 - val_accuracy: 0.7667
Epoch 8/10
18

In [61]:
test_loss, test_acc = model.evaluate(np_test_images,  np_test_labels, verbose=2)

19/19 - 1s - loss: 0.0849 - accuracy: 0.9864


In [62]:
base_model.trainable = True
print("original layers in the base model before fine tuning: ", len(base_model.layers))
fine_tune = 100
for layer in base_model.layers[:fine_tune]:
  layer.trainable =  False

original layers in the base model before fine tuning:  132


In [63]:
print("trainable variables after fine tuning: ",len(model.trainable_variables))

trainable variables after fine tuning:  43


In [64]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, 3, 3, 2048)        20861480  
_________________________________________________________________
dropout_2 (Dropout)          (None, 3, 3, 2048)        0         
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 1536)              3147264   
_________________________________________________________________
dense_2 (Dense)              (None, 62)                95294     
Total params: 24,104,038
Trainable params: 12,720,902
Non-trainable params: 11,383,136
_________________________________

In [65]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [66]:
history = model.fit(X_train,y_train,epochs=20,validation_data=(X_valid,y_valid),initial_epoch=history.epoch[-1])

Epoch 10/20
18/18 [==============================] - 5s 142ms/step - loss: 2.5425 - accuracy: 0.5031 - val_loss: 3.0250 - val_accuracy: 0.7000
Epoch 11/20
18/18 [==============================] - 1s 37ms/step - loss: 0.4209 - accuracy: 0.8848 - val_loss: 2.1402 - val_accuracy: 0.7000
Epoch 12/20
18/18 [==============================] - 1s 37ms/step - loss: 0.2129 - accuracy: 0.9442 - val_loss: 3.1343 - val_accuracy: 0.7667
Epoch 13/20
18/18 [==============================] - 1s 37ms/step - loss: 0.0475 - accuracy: 0.9860 - val_loss: 2.3237 - val_accuracy: 0.7333
Epoch 14/20
18/18 [==============================] - 1s 37ms/step - loss: 0.1626 - accuracy: 0.9538 - val_loss: 1.4805 - val_accuracy: 0.7667
Epoch 15/20
18/18 [==============================] - 1s 37ms/step - loss: 0.1126 - accuracy: 0.9687 - val_loss: 2.4353 - val_accuracy: 0.7667
Epoch 16/20
18/18 [==============================] - 1s 37ms/step - loss: 0.1934 - accuracy: 0.9406 - val_loss: 3.1472 - val_accuracy: 0.8000
Epoch

In [67]:
test_loss, test_acc = model.evaluate(np_test_images,  np_test_labels, verbose=2)

19/19 - 0s - loss: 1.1385 - accuracy: 0.9284


In [68]:
predict = np.argmax(model.predict(np_test_images), axis = 1)

In [69]:
labelCon = []
for i in range(62):
  labelCon.append(i)

In [70]:
from sklearn.metrics import classification_report, confusion_matrix
print('Confusion Matrix')
print(confusion_matrix(np_test_labels, predict , labelCon))

Confusion Matrix
[[  9   6   0 ...   0   0   0]
 [  0 110   0 ...   0   0   0]
 [  0   6   7 ...   0   0   0]
 ...
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]]


In [71]:
print('Classification Report')
print(classification_report(np_test_labels, predict , labelCon))

Classification Report
              precision    recall  f1-score   support

           0       1.00      0.60      0.75        15
           1       0.75      1.00      0.86       110
           2       1.00      0.54      0.70        13
           3       0.94      1.00      0.97        15
           4       1.00      0.80      0.89        15
           5       1.00      0.91      0.95        11
           6       1.00      0.94      0.97        18
           7       0.99      1.00      1.00       157
           8       1.00      1.00      1.00        27
           9       0.91      0.56      0.69        18
          10       1.00      0.90      0.95        21
          11       0.78      1.00      0.88         7
          12       1.00      1.00      1.00        18
          13       1.00      0.86      0.92        90
          14       1.00      0.95      0.98        43
          15       1.00      1.00      1.00         9
          16       0.00      0.00      0.00         0
     

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
